In [ ]:
# fake_news_gradio_app.py
!pip install gradio
# --- Imports ---
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import gradio as gr # Import Gradio library

# --- Configuration (Models) ---
GENERATION_MODEL_NAME = "gpt2-medium"
DETECTION_MODEL_NAME =  "mrm8488/bert-tiny-finetuned-fake-news-detection"

# --- Load Models (Cached by Hugging Face) ---
# These functions will load the models once when the Gradio app starts.
# Gradio handles caching automatically, similar to Streamlit's @st.cache_resource.

def load_generation_pipeline():
    """Loads the text generation pipeline."""
    print(f"Loading generation model: {GENERATION_MODEL_NAME}")
    return pipeline("text-generation", model=GENERATION_MODEL_NAME)

def load_detection_pipeline():
    """Loads the fake news detection pipeline."""
    print(f"Loading detection model: {DETECTION_MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(DETECTION_MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(DETECTION_MODEL_NAME)
    return pipeline("text-classification", model=model, tokenizer=tokenizer)

# Initialize pipelines globally so they are loaded once
generator = load_generation_pipeline()
detector = load_detection_pipeline()

# --- Core Logic Functions for Gradio Interface ---

def generate_news_func(topic: str) -> str:
    """Generates news based on the given topic."""
    if not topic:
        return "Please enter a topic to generate news."

    prompt = f"a shocking news report just came in regarding {topic}. Sources say that"
    generated_result = generator(prompt, max_new_tokens=100, num_return_sequences=1, truncation=True)
    generated_news = generated_result[0]['generated_text']
    return generated_news
    